In [10]:
import glob
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [55]:
import spotipy
import time
import requests
from spotipy.oauth2 import SpotifyClientCredentials 
spotify_client_id = '1746dbf63fa84b4095f88aa11df2eda2'
spotify_client_secret = '4e0ae57d50be40238cff2a9ef16e31fd'
spotify_redirect = 'http://localhost:8888/callback'
spotify_scope = 'user-top-read'
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret))

In [56]:
songs_list = []

years = ['1995','1996','1997','1998','1999','2000','2001',
              '2002','2003','2004','2005','2006','2007','2008',
              '2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'
              ,'2019','2020','2021','2022']
              
for year in years : 
    for i in range(0,500,50):
        track_results = sp.search(q='year:'+year,type='track',limit=50, offset=i)
        for i, t in enumerate(track_results['tracks']['items']):
            artist = sp.artist(t["artists"][0]["external_urls"]["spotify"])
            album = sp.album(t["album"]["external_urls"]["spotify"])
            songs_data = {
                    'id':t['id'],
                    'song name':t['name'],
                    'artist name':t['artists'][0]['name'],
                    'artist popularity' : artist['popularity'],
                    'artist genres':artist["genres"],
                    'album genres': album["genres"],
                    'release_date':t['album']['release_date'],
                    'song duration':t['duration_ms'],
                    'song popularity' : t['popularity'],
                }
            songs_list.append(songs_data)
df_songs = pd.DataFrame(songs_list)

df_songs = pd.DataFrame(songs_list)


KeyboardInterrupt: ignored

#Importing Packages

In [53]:
df_songs.to_csv("Songs_Data.csv")
df_songs

,id,song name,artist name,artist popularity,artist genres,album genres,release_date,song duration,song popularity
0,1DIXPcTDzTj8ZMHt3PDt8p,Gangsta's Paradise,Coolio,70,"[g funk, gangster rap, hip hop, rap, west coas...",[],1995-11-07,240693,87
1,5QLHGv0DfpeXLNFo7SFEy1,1979 - Remastered 2012,The Smashing Pumpkins,71,"[alternative metal, alternative rock, grunge, ...",[],1995,266200,76
2,6GtX0jaNL8IjVQfrDBx81z,Bullet With Butterfly Wings - Remastered 2012,The Smashing Pumpkins,71,"[alternative metal, alternative rock, grunge, ...",[],1995,258466,74
3,6iy4PoAuZBMvtrlDX4VxC7,Check Yes Or No,George Strait,71,"[contemporary country, country, country road, ...",[],1995-09-12,200666,70
4,6xkryXuiZU360Lngd4sx13,Fantasy,Mariah Carey,77,"[dance pop, pop, r&b, urban contemporary]",[],1995-10-03,243493,75
...,...,...,...,...,...,...,...,...,...
2795,1PckUlxKqWQs3RlWXVBLw3,About Damn Time,Lizzo,78,"[escape room, minnesota hip hop, pop, trap queen]",[],2022-04-14,191822,87
2796,5JhUm1v5AbI5Yu6jLKCPRO,No Se Va - En Vivo,Grupo Frontera,73,[nueva musica mexicana],[],2022-04-28,194019,89
2797,2TSfStvlAMLw89u3tali79,Dah Dah DahDah,Nardo Wick,73,"[florida rap, rap, trap, viral rap]",[],2022-07-22,151776,79
2798,28n8z9IQKyjMPxmvYMBqZa,The Kind of Love We Make,Luke Combs,80,"[contemporary country, country]",[],2022-06-17,224426,80


In [25]:
def auth_spotify (client_id, client_secret):
    URl= 'https://accounts.spotify.com/api/token'

    auth_response = requests.post(URl, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    
    auth_response= auth_response.json()
    access_token= auth_response['access_token']
    
    return access_token

In [26]:
client_id= '762d2685f9a141baa0e1a74666577274'
client_secret= '3beff23b97c7466b9bd0dc880b965dc7'
access_token= auth_spotify(client_id, client_secret)

In [27]:
def API_search_request(keywords, search_type, results_limit, results_offset, ltrack, song_ids, artist_ids, album_ids):

    off = str(results_offset)
    lim = str(results_limit)
    header= {'Authorization': "Bearer {}".format(access_token)}

    url = 'https://api.spotify.com/v1/search?q=year:'+ keywords +'&type=' + search_type +'&offset='+ off +'&limit=' + lim

    r = requests.get(url, headers = header)

    if r: 
       j = r.json()
    else:
      return r


    litem = j['tracks']['items']
    try:
        for l in litem:
        
            if l['id'] not in song_ids:
                song_ids.append( l['id'] )

            if l['artists'][0]['id'] not in artist_ids:
                artist_ids.append( l['artists'][0]['id'] )

            if l['album']['id'] not in album_ids:
                album_ids.append(  l['album']['id'] )
        
        
            k =   [  l['popularity'],
        
                     l['id'], 
                     l['artists'][0]['id'],
                     l['album']['id'],

                     l['name'],
                     l['artists'][0]['name'],
                     l['album']['name'],

                     l['explicit'], 
                     l['disc_number'],
                     l['track_number']]
        
            ltrack.append(k)
    except:
         ValueError

In [28]:
def API_get_audio_feature(songids, audioF):
    
    #print(songids)
    #print '>> call art several'
    track_ids = ','.join(songids)

    url = 'https://api.spotify.com/v1/audio-features?ids=' + track_ids   
    header= {'Authorization': "Bearer {}".format(access_token)}
    
    r = requests.get(url, headers=header)
    
    if r: 
      j = r.json()
    else:
      return r
    
    # print(j)
    ll = j['audio_features']

    try:

        for l in ll:
            k =  [  l['id'],l['uri'],
                    l['tempo'],l['type'],
                    l['key'],l['loudness'],
                    l['mode'],l['speechiness'],
                    l['liveness'],l['valence'],
                    l['danceability'],l['energy'],
                    l['track_href'],l['analysis_url'],
                    l['duration_ms'],l['time_signature'],
                    l['acousticness'],l['instrumentalness'] ]

            audioF.append(k)
        
    except:
        ValueError
    
        

In [29]:
def API_get_albums(album_ids, album_data):
   

    alb_ids = ','.join(album_ids)

    url = 'https://api.spotify.com/v1/albums?ids=' + alb_ids
    header= {'Authorization': "Bearer {}".format(access_token)}
    r = requests.get(url, headers= header)

    if r:
       j = r.json()
    else:
       return r


    ll = j['albums']
    
    try:
        for l in ll:
            k = [  l['id'], 
                   l['genres'],
                   l['popularity'],
                   l['release_date'] ]

            album_data.append(k)
    
    except:
        ValueError

In [30]:
def API_get_artists(artist_ids, artist_data):

    art_ids = ','.join(artist_ids)

    url = 'https://api.spotify.com/v1/artists?ids=' + art_ids

    r = requests.get(url, headers = {'Authorization': "Bearer {}".format(access_token)})

    if r:
       j = r.json()
    else:
       #print 'for this specific art_ids, response reaches maximum, return'
       return r

    
    ll = j['artists']

    try:
        for l in ll:
        
            k = [  l['id'], 
                   l['genres'],
                   l['popularity'] ]

            artist_data.append(k)
    
    except:
        ValueError
    

In [42]:
years = ['1995','1996','1997','1998','1999','2000','2001',
              '2002','2003','2004','2005','2006','2007','2008',
              '2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'
              ,'2019','2020','2021','2022']
              
    #queries = ['2017']
    # Query and request from API are different!
    # Number of track query need to make
num_tracks_per_query = 10000

for query in years:
        
    ltrack = []
    song_ids = []
    artist_ids = []
    album_ids = []
    
    audioF = []
    artist_data = []
    album_data = []

        
    col1 = [   'popularity',
                'song_id',     'artist_id',     'album_id', 
                'song_name',   'artist_name',   'album_name', 
                'explicit',    'disc_number',   'track_number']
    
    
    col2 =  [  'song_id', 'uri',
                'tempo', 'type',
                'key', 'loudness',
                'mode', 'speechiness',
                'liveness', 'valence',
                'danceability', 'energy',
                'track_href', 'analysis_url',
                'duration_ms', 'time_signature',
                'acousticness', 'instrumentalness' ]
    
    col3 =  [  'artist_id',  'artist_genres',  'artist_popularity']
    
    col4 =  [  'album_id',  'album_genres',   'album_popularity',  'album_release_date']
    
    n = 0 
    idx = 0
        
    while idx < num_tracks_per_query:  
        
      API_search_request(query, 'track', 50, idx, ltrack, song_ids, artist_ids, album_ids)   
      n +=1
      idx += 50 
      # Limit API requests to at most 3ish calls / second
      time.sleep(0.3)                                    
        
      for idx in range(0, len(song_ids), 50):
        API_get_audio_feature(song_ids[idx: idx+50], audioF)
        time.sleep(0.3)
        
      for idx in range(0, len(artist_ids), 50):
        API_get_artists(artist_ids[idx: idx+50], artist_data)
        time.sleep(0.3)
        
      for idx in range(0, len(album_ids), 20):
        API_get_albums(album_ids[idx: idx+20], album_data)
        time.sleep(0.3)    

        
        
      df1 = pd.DataFrame(ltrack, columns=col1)
      
      df2 = pd.DataFrame(audioF, columns=col2) 
      
      
      df3 = pd.DataFrame(artist_data, columns=col3)
      
      df4 = pd.DataFrame(album_data, columns=col4)
      
      df = df1.merge(df2, on='song_id', how='outer').merge(df3, on='artist_id', how='outer').merge(
            df4, on='album_id', how='outer')
      
      filename = query + '.csv'                      
      
      df.to_csv(filename, sep='\t')
      print (df.shape)
      print("Length = "  + str(len(df.index)))
      if (len(df.index) > 10000) : break
      
     

(50, 32)
Length = 50
(681, 32)
Length = 681
(2963, 32)
Length = 2963
(8201, 32)
Length = 8201
(19091, 32)
Length = 19091
(50, 32)
Length = 50
(684, 32)
Length = 684
(2992, 32)
Length = 2992
(9220, 32)
Length = 9220
(22776, 32)
Length = 22776
(50, 32)
Length = 50
(667, 32)
Length = 667
(2858, 32)
Length = 2858
(7874, 32)
Length = 7874
(18302, 32)
Length = 18302
(50, 32)
Length = 50
(682, 32)
Length = 682
(2960, 32)
Length = 2960
(8271, 32)
Length = 8271
(19520, 32)
Length = 19520
(50, 32)
Length = 50
(672, 32)
Length = 672
(2893, 32)
Length = 2893
(8053, 32)
Length = 8053
(18951, 32)
Length = 18951
(50, 32)
Length = 50
(669, 32)
Length = 669
(2904, 32)
Length = 2904
(8146, 32)
Length = 8146
(19296, 32)
Length = 19296
(50, 32)
Length = 50
(551, 32)
Length = 551
(2438, 32)
Length = 2438
(7321, 32)
Length = 7321
(18146, 32)
Length = 18146
(50, 32)
Length = 50
(675, 32)
Length = 675
(2927, 32)
Length = 2927
(8187, 32)
Length = 8187
(19319, 32)
Length = 19319
(50, 32)
Length = 50
(545, 32)
L